In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from sklearn.metrics import classification_report
import pandas as pd
import idx2numpy
from sklearn.metrics import confusion_matrix
from keras.models import load_model
from sklearn.metrics import accuracy_score
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
def read_data(path):
    file_data = path + 'input/train-images-idx3-ubyte'
    arr_data = idx2numpy.convert_from_file(file_data)
    arr_data = arr_data.reshape(60000, 28 * 28)
    file_label = path + 'input/train-labels-idx1-ubyte'
    arr_label = idx2numpy.convert_from_file(file_label)
    file_test = path + 'input/t10k-images-idx3-ubyte'
    test_data = idx2numpy.convert_from_file(file_test)
    test_data = test_data.reshape(10000, 28 * 28)
    test_lab = path + 'input/t10k-labels-idx1-ubyte'
    test_label = idx2numpy.convert_from_file(test_lab)
    return arr_data, arr_label, test_data, test_label


def KerasCNN(path):
    # Load the dataset files and pre-process data
    train_x, train_y, test_x, test_y = read_data(path)
    train_x = train_x.reshape([-1, 28, 28, 1]) / 255.0
    train_X = train_x[:50000]
    Valid_x = train_x[50000:]
    train_Y = train_y[:50000]
    Valid_y = train_y[50000:]

    # Build CNN classifier based on Keras
    model = keras.Sequential([
        # (-1,28,28,1)->(-1,28,28,32)
        keras.layers.Conv2D(input_shape=(28, 28, 1), filters=32, kernel_size=5, strides=1, padding='same'),
        # Padding method),
        # (-1,28,28,32)->(-1,14,14,32)
        keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'),

        keras.layers.Dropout(0.2),
        # (-1,14,14,32)->(-1,14,14,64)
        keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),  # Padding method),
        # (-1,14,14,64)->(-1,7,7,64)
        keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'),

        keras.layers.Dropout(0.2),
        # (-1,7,7,64)->(-1,7*7*64)
        keras.layers.Flatten(),
        # (-1,7*7*64)->(-1,256)
        keras.layers.Dense(256, activation=tf.nn.relu),
        # (-1,256)->(-1,10)
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    print(model.summary())

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_X,
                        train_Y,
                        validation_data=(Valid_x, Valid_y),
                        epochs=10,
                        batch_size=64,
                        verbose=1
                        )
    model.save("my_model.h5")

    # Print test accuracy and loss line chart
    accuracy = history.history['acc']
    val_accuracy = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(accuracy))

    plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    return model


def Loadmodel(filename):
    model = load_model(filename)
    return model

def makeprediction(path,model):
    train_x, train_y, test_x, test_y = read_data(path)
    test_x = test_x.reshape([-1, 28, 28, 1]) / 255.0
    classNames = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                  'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    # Print precision, recall and f1-score
    predicted_classes = model.predict_classes(test_x)
    print(classification_report(test_y, predicted_classes, target_names=classNames))  # KNN分类
    # Print confusion matrix
    con_array = confusion_matrix(test_y, predicted_classes, labels=range(10))
    con_array = con_array / con_array.sum(axis=1)
    df_cm = pd.DataFrame(con_array, range(10), range(10))
    sns.set(font_scale=0.8)
    snsimage = sns.heatmap(df_cm, annot=True, annot_kws={"size": 8})
    figure = snsimage.get_figure()
    figure.savefig("output.png", dpi=400)
    accuracy = accuracy_score(test_y,predicted_classes)
    return accuracy

### Train model and make prediction

In [ ]:
# change path to your local path
path = ""
model = KerasCNN(path)
makeprediction(path,model)

### Load saved model and make prediction

In [ ]:
# add your local path to your filename
filename = "my_model.h5"
model =Loadmodel(filename)
# change path to your local path
path = ""
makeprediction(path,model)